This is my practice during the #USAID_pause
Trying to make a user access way to view the DHS data directly through the website instead of going through Stata
Step 1: Access the API
Step 2: Build out how to get and wrangle data
Step 3: Develop the user interface to allow people to make changes on the data
Step 4: ???


In [29]:
import requests
import pandas as pd

import os
import json
from urllib.request import urlopen
BASE_URL = "https://api.dhsprogram.com/rest/dhs/data"  

def fetch_dhs_data(country_codes):
    # Define the API parameters
    params = {
        "country_ids": ",".join(country_codes),
        "indicator_ids": "ML_ITNA_P_ACC,ML_NETU_P_ITN",  # Hypothetical indicator IDs
    }
    
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        # Process the response
        data = response.json()  # Assuming JSON format from the API
        return pd.DataFrame(data)  # Convert to pandas DataFrame
    else:
        print("Error fetching data:", response.status_code)
        return None

In [30]:
import shiny
import plotly.express as px
import requests
import pandas as pd


# Create the shiny UI
app_ui = ui.page_fluid(
    ui.input_selectize("countries", "Select Countries", choices=["Kenya", "Nigeria", "Ghana"], multiple=True),
    ui.output_plot("chart"),
)

# Define the server function
def server(input, output, session):

    @output()
    @render.plot
    def chart():
        selected_countries = input.countries()
        
        # Fetch the data for the selected countries
        data = fetch_dhs_data(selected_countries)
        
        if data is not None:
            # Plot the data
            fig = px.line(data, x="SurveyYear", y=["ML_ITNA_P_ACC,ML_NETU_P_ITN"], 
                          title="ITN Indicators Over Time")
            return fig
        else:
            return None

# Create the Shiny app
app = shiny.App(app_ui, server)



AttributeError: module 'shiny' has no attribute 'jupyter'

: 